In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import urllib.parse
import pymongo
import pprint
import pandas as pd

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.stocks
collection = db.mid_cap

In [4]:
url = 'https://www.barchart.com/stocks/most-active/price-volume-leaders?screener=mid_cap'
response = requests.get(url)
print(response.status_code)

403


In [5]:
# searchterm = urllib.parse.quote('left bold plusIconTd elp')
# print(searchterm)

In [6]:
# SCRAPING 
# Set the executable path and initialize the chrome browser in splinter
executable_path = {"executable_path":"chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)
browser.visit("https://www.barchart.com/stocks/most-active/price-volume-leaders?screener=mid_cap")

In [7]:
# Convert the browser html to a soup object 
soup = browser.html
stock_soup = BeautifulSoup(soup, 'html.parser')

In [8]:
#print(stock_soup.prettify())

In [9]:
stock_symbol = stock_soup.find("td", class_= "symbol text-left").text
print(stock_symbol)

         SDC     


In [10]:
stock_name = stock_soup.find("td", class_= "symbolName text-left").text
print(stock_name)

       Smiledirectclub Inc      


In [11]:
stock_price = stock_soup.find("td", class_= "lastPrice").text
print(stock_price)

       11.70      


In [12]:
stock_volume = stock_soup.find("td", class_= "volume").text
print(stock_volume)

       33,561,199      


In [13]:
stock_change = stock_soup.find("td", class_= "percentChange up").text
print(stock_change)

       +14.71%      


In [14]:
stock_symbol_all = stock_soup.find_all("td", class_= "symbol text-left")
stock_name_all = stock_soup.find_all("td", class_= "symbolName text-left")
stock_price_all = stock_soup.find_all("td", class_= "lastPrice")
stock_volume_all = stock_soup.find_all("td", class_= "volume")
stock_change_all = stock_soup.find_all("td", class_= "percentChange up")


# print(stock_symbol_all)

# print(stock_name_all)

# print(stock_price_all)

# print(stock_volume_all)

# print(stock_change_all)

In [15]:
symbol =[]
for stock in stock_symbol_all:
    #print(stock.text)
    symbol_list = symbol.append(stock.text)
    #print(symbol)

In [16]:
price =[]
for stock in stock_price_all:
    #print(stock.text)
    price_list = price.append(stock.text)
    #print(price)

In [17]:
volume = []   
for vol in stock_volume_all:
    #print(vol.text)
    volume_list = volume.append(vol.text)
    #print(volume)

In [18]:
name = []   
for names in stock_name_all:
    #print(names.text)
    names_list = name.append(names.text)
    #print(name)

In [19]:
change = []   
for names in stock_change_all:
    #print(names.text)
    change_list = change.append(names.text)
    #print(change)

In [20]:
# Convert to Pandas dataframe and export in csv
df = pd.DataFrame({'stock_name':name, 'stock_symbol':symbol,'stock_price':price, 'stock_volume':volume})
df

,stock_name,stock_symbol,stock_price,stock_volume
0,Smiledirectclub Inc,SDC,11.70,"33,561,199"
1,Tilray Inc,TLRY,21.22,"14,194,400"
2,Restoration Hardware Holdings,RH,223.36,"1,345,691"
3,Shake Shack Inc,SHAK,64.10,"4,263,853"
4,Nio Inc,NIO,3.76,"55,220,619"
5,Enphase Energy Inc,ENPH,30.20,"6,406,500"
6,Nektar Therapeutics,NKTR,27.96,"6,736,300"
7,United States Steel Corp,X,10.91,"16,742,542"
8,STI Inc,STAG,31.50,"4,791,249"
9,Cloudera Inc,CLDR,11.58,"12,512,360"


In [21]:
# Importing the dataframe into mongodb

data = df.to_dict(orient='records')  # Here's our added param..

collection.insert_many(data)

In [22]:
df.to_csv('stocks_mid_cap.csv', index=False, encoding='utf-8')